In [1]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Literal
from langchain_ollama import ChatOllama
from dotenv import load_dotenv
from pydantic import BaseModel, Field


In [2]:
load_dotenv()

False

In [3]:
model = ChatOllama(
    model="gemma:2b"
)

In [4]:
class SentimentSchema(BaseModel):
    
    sentiment : Literal['positive', 'negative'] = Field(description="The sentiment of the review")
    

In [5]:
structured_model = model.with_structured_output(SentimentSchema)

In [7]:
prompt = """What is the sentiment of the following review?/n The product was great, I really liked it!"""
structured_model.invoke(prompt)

SentimentSchema(sentiment='positive')

In [8]:
class ReviewState(TypedDict):
    
    review: str
    sentiment: Literal['positive', 'negative']
    diagnosis: str
    response: str

In [14]:
def find_sentiment(state: ReviewState) -> ReviewState:
    
    prompt = f"What is the sentiment of the following review? {state['review']}"
    sentiment = structured_model.invoke(prompt).sentiment
    
    return {'sentiment': sentiment}


def check_condition(state : ReviewState) -> Literal["postive_response","run_diagnosis"]:
    
    if state['sentiment'] == 'positive':
        return 'postive_response',
    else :
        return 'run_diagnosis'
    

def run_diagnosis(state: ReviewState) -> ReviewState:
    
    prompt = f"Diagnose the following negative review in aspect of tone , issue and urgency \n: {state['review']}"
    diagnosis = model.invoke(prompt).content
    
    return {'diagnosis': diagnosis}


def postive_response(state: ReviewState) -> ReviewState:

    prompt = f"Write a positive response and Thank you note for the following review: {state['review']}"
    response = model.invoke(prompt).content

    return {'response': response}

def negative_response(state: ReviewState) -> ReviewState:
    
    prompt = f"Write a apology note and positive solution of the issue based on diagnosis: {state['diagnosis']}"
    
    response = model.invoke(prompt).content
    
    return {'response': response}
    

In [17]:
graph = StateGraph(ReviewState)

graph.add_node('find_sentiment', find_sentiment)
graph.add_node('run_diagnosis', run_diagnosis)
graph.add_node('postive_response', postive_response)
graph.add_node('negative_response', negative_response)


graph.add_edge(START, 'find_sentiment')
graph.add_conditional_edges('find_sentiment', check_condition)
graph.add_edge('run_diagnosis', 'negative_response')
graph.add_edge('postive_response', END)
graph.add_edge('negative_response', END)

workflow = graph.compile()

In [22]:
initial_state = {
    'review': "The product was too bad , I need to return it please how can I do that?",}

workflow.invoke(initial_state)

{'review': 'The product was too bad , I need to return it please how can I do that?',
 'sentiment': 'negative',
 'diagnosis': "**Tone:**  Negative\n\n**Issue:** Product quality \n\n**Urgency:** High\n\n**Explanation:** \n\n* **Tone:**  The tone is negative and accusatory, with the reviewer expressing strong dissatisfaction with the product and its quality. \n\n\n* **Issue:** The issue is with the product itself, and not with the customer's handling or use of it. \n\n\n* **Urgency:**  The review is an urgent matter, as the reviewer is looking to get a refund for a product that did not meet their expectations.",
 'response': "**Apology Note:**\n\nDear [Company Name],\n\nI am writing to express my sincere apologies for the negative feedback I received regarding the [Product Name]. I understand that the product did not meet your expectations and that this has caused you great disappointment.\n\nI take full responsibility for the quality of the product and the customer service I provided. I